# Simulação de um Sistema Solar com um Vulcano

## Criação de um dataframe para o Vulcano

In [8]:
# Import modules
import os
import yaml
import pandas as pd
import numpy as np
from oe2pv import orbel_el2xv

## Vetorizando a função de transformação de elementos orbitais

In [9]:
# Transforms orbel_el2xv in vectorized function
orbel_el2xv_vec = np.vectorize(orbel_el2xv)

## Definindo funções

In [10]:
def read_config_file(config_file):
    """
    Read config file and go to home directory
    """
    home_dir = os.path.dirname(os.path.abspath(config_file))
    os.chdir(home_dir)

    # Read yaml file
    with open(config_file, 'r') as stream:
        try:
            config = yaml.load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    
    return config

## Lendo arquivos de configuração e definições de variáveis

In [11]:
config = read_config_file("config.yaml")

# Read variable in config file
vulcans_variants = len(config["vulcans_semi_axis"])
vulcans_clones = config["vulcans_clones"]

# Mass of the Sum [kg]
mass_sun_kg = config["mass_sun_kg"]

# Mass of the Sun, considering G = 1
mass_sun_grav = config["mass_sun_grav"]

# Conic section is ellipse # Constant used in oe2pv function
ialpha = config["ialpha"]

# Gravitational factor of the Sun
gm = config["gm"]

# Semi-axis
a_variants = config["a_variants"]

## Criando o dataframe

In [12]:
# ### Create dataframe

# Initial dataframe
vulcan = pd.DataFrame(config["vulcan"], index=[0])

# Create gravitational mass
vulcan['mass_grav'] = vulcan['mass'] * mass_sun_grav / mass_sun_kg

# Create gmpl column
vulcan['gmpl'] = vulcan['mass_grav'] + gm

# Replicate initial values in each simulate
vulcan = vulcan.append([vulcan] * (vulcans_variants *\
                            vulcans_clones - 1), ignore_index=True)

# Create semi-axis
vulcan['a'] = np.repeat(a_variants, vulcans_clones)

# Create period
vulcan["period"] = np.sqrt(((4 * np.pi**2) / mass_sun_grav)* vulcan['a'] **3)

# Create random eccentricity
# Usin numpy.random.ranf. For range = (a,b): (b - a) * random_sample() + a
vulcan['e'] = 0.01 * np.random.ranf((vulcans_variants * vulcans_clones,))

# Create random inclination
# Usin numpy.random.ranf. For range = (a,b): (b - a) * random_sample() + a
vulcan['inc'] = np.deg2rad(0.01 * np.random.ranf((vulcans_variants * vulcans_clones,)))


# Create capom angle
vulcan['capom'] = np.deg2rad(np.random.randint(0, 361, vulcans_variants * vulcans_clones))

# Create omega angle
vulcan['omega'] = np.deg2rad(np.random.randint(0, 361, vulcans_variants * vulcans_clones))

# Create M angle - Mean Anomaly
vulcan['capm'] = np.deg2rad(np.random.randint(0, 361, vulcans_variants * vulcans_clones))

# Create postions and velocities 

x, y, z, vx, vy, vz = orbel_el2xv_vec(vulcan["gmpl"], ialpha,vulcan["a"],\
                                      vulcan["e"],vulcan["inc"],\
                                      vulcan["capom"], vulcan["omega"],\
                                      vulcan["capm"])

for j in ['x', 'y', 'z', 'vx', 'vy', 'vz']:
        exec("vulcan['{0}'] = {0} ".format(j))

## Salvando o dataframe

In [13]:
# Save vulcan dataframe
vulcan.to_csv("vulcan.csv", index=False)

## Verificando o dataframe criado
Neste caso, é necessário ver a interface entre a mudança de semieixo maior

In [14]:
read_vulcano = pd.read_csv("vulcan.csv")
print(read_vulcano[5:15])

            mass     radio     mass_grav      gmpl     a     period         e  \
5   5.972300e+24  6378.137  8.887539e-10  0.000296  0.05   4.083684  0.006083   
6   5.972300e+24  6378.137  8.887539e-10  0.000296  0.05   4.083684  0.004413   
7   5.972300e+24  6378.137  8.887539e-10  0.000296  0.05   4.083684  0.004092   
8   5.972300e+24  6378.137  8.887539e-10  0.000296  0.05   4.083684  0.001638   
9   5.972300e+24  6378.137  8.887539e-10  0.000296  0.05   4.083684  0.000537   
10  5.972300e+24  6378.137  8.887539e-10  0.000296  0.10  11.550403  0.008694   
11  5.972300e+24  6378.137  8.887539e-10  0.000296  0.10  11.550403  0.002526   
12  5.972300e+24  6378.137  8.887539e-10  0.000296  0.10  11.550403  0.005805   
13  5.972300e+24  6378.137  8.887539e-10  0.000296  0.10  11.550403  0.002429   
14  5.972300e+24  6378.137  8.887539e-10  0.000296  0.10  11.550403  0.005104   

         inc     capom     omega      capm         x         y             z  \
5   0.000105  4.520403  1.01